Questo documento finalizza i word embedding utilizzando dimensionalità 10, CBOW (che funziona meglio per grandi corpus) e soprattutto tiene conto delle modifiche di gennaio alla tokenizzazione (rimozione parole inutili, stemming ecc)

In [1]:
import json
import pandas as pd
import time
import os
from tqdm.notebook import tqdm
DIR = "C:/Users/enduser/OneDrive - Politecnico di Milano/Ingegneria matematica/Tesi/ProveDiCodice/E3C-Corpus/data_collection/Italian/layer3"

# Elenco dei file nella directory
data_files = os.listdir(DIR)

In [2]:
dfs = []
for filename in tqdm(os.listdir(DIR)):
    authors_string = False
    authors_dict = False
    f = os.path.join(DIR, filename)
    if os.path.isfile(f):
      with open(f, 'r', encoding='utf-8') as fp: # QUESTA è LA RIGA MODIFICATA, VEDI CODICE ORIGINALE VITTORIO PER CAPIRE, QUESTA MODIFICA è STATA NECESSARIA ALTRIMENTI MI DAVA ERRORE (SU COLLAB NO)
        d = json.load(fp)
      if d['authors'] == []:
        d['authors'] = ''
        i = [0]
      elif isinstance(d['authors'], str): #for some Spanish documents (es ES102568.json)
        authors_string = True
        i = [0]
      elif isinstance(d['authors'], dict): #for Basque
        authors_dict = True
        i = [0]
      else:
        i = list(range(len(d['authors'])))
      data = pd.DataFrame(d, index=i)
      if d['authors'] != '':
        try:
          data.authors = pd.DataFrame(data.authors.values.tolist())['author']
        except: #spanish has different format
          if authors_dict:
            data.authors = d['authors']['author']
          elif not authors_string:
            data.authors = pd.DataFrame(data.authors.values.tolist())[1]
        cols = list(data.columns) # columns are different in different languages
        cols.remove('authors')
        data = data.groupby(cols)['authors'].apply(','.join).reset_index()
      dfs.append(data) # append the data frame to the list
df = pd.concat(dfs, ignore_index=True, axis=0) # concatenate all the data frames in the list.

  0%|          | 0/10213 [00:00<?, ?it/s]

In [10]:
df.loc[9629]

doi                                                                  
publication_date                                           2021-02-01
id                                                           IT111703
url                                                                  
source                                   Agenzia Italiana del Farmaco
source_url                      https://farmaci.agenziafarmaco.gov.it
licence                                                      CC-BY-NC
language                                                           it
type                                                            other
description                patient information leaflets for medicines
text                Confezioni \r\n Ultiva ev 5 flaconi 2 mg (conf...
authors                                                              
Name: 9629, dtype: object

In [11]:
df.loc[8368]

doi                                                                  
publication_date                                           2021-02-01
id                                                           IT110442
url                                                                  
source                                   Agenzia Italiana del Farmaco
source_url                      https://farmaci.agenziafarmaco.gov.it
licence                                                      CC-BY-NC
language                                                           it
type                                                            other
description                patient information leaflets for medicines
text                Confezioni \r\n Remifentanil Mylan Generics 1 ...
authors                                                              
Name: 8368, dtype: object

In [17]:
df.loc[2124]

doi                                                                  
publication_date                                           2004-MM-DD
id                                                           IT104198
url                      http://www.mattiolifp.it/Casi_ita/Caso_5.pdf
source                              Prof. Dr. Francescopaolo Mattioli
source_url                                   http://www.mattiolifp.it
licence                                                         CC-BY
language                                                           it
type                                                            other
description                                                          
text                Femmina - aa. 77 –\n\nAnamnesi patologica remo...
authors                                       Francescopaolo Mattioli
Name: 2124, dtype: object



# NB: gensim richiede in input le frasi, non corpus, per questo l'input è "sentences"

In [5]:
import pickle

In [6]:
with open('corpusParoleInutiliStemming.pkl', 'rb') as file:
    # Carica i dati dal file
    corpus_filtrato = pickle.load(file) # corpus filtrato è il corpus senza le parole inutili

In [9]:
corpus_filtrato

[['iniz',
  'far',
  'dialis',
  'età',
  '17',
  'anni',
  'centr',
  'assistt',
  'limit',
  'cal',
  'prim',
  'anni',
  'ottant',
  'pung',
  'sol',
  'access',
  'vascol',
  'dop',
  'otto',
  'anni',
  'frequenz',
  'cal',
  'ven',
  'trapiant',
  'dop',
  'diciannov',
  'anni',
  'trapiant',
  'rientr',
  'cal',
  'sottoscritt',
  'sempr',
  'stat',
  'cert',
  'sinton',
  'generazional',
  'rientr',
  'vis',
  'centr',
  'bruxelles',
  'raccolt',
  'interess',
  'dat',
  'evidt',
  'pensa',
  'possibil',
  'candid',
  'implement',
  'puntur',
  'occhiell',
  'fistol',
  'fav',
  'distal',
  'sinistr',
  'sit',
  'puntur',
  'arter',
  'posizion',
  'met',
  'avambracc',
  'anterior',
  'central',
  'sit',
  'puntur',
  'venos',
  'alloc',
  'cefal',
  'estern',
  'bracc',
  'circ',
  '7',
  'cm',
  'pieg',
  'gom',
  'legger',
  'lateralizz',
  'vas',
  'present',
  'già',
  'po',
  'dilat',
  'sebben',
  'pot',
  'ancor',
  'defin',
  'aneurismat',
  'livell',
  'anastom',
  '

In [13]:
from gensim.models import Word2Vec


# Definire il modello Word2Vec con l'approccio CBOW
model = Word2Vec(sentences=corpus_filtrato, vector_size=10, window=8, min_count=1, workers=4, sg=0) #sg = 0  CBOW 1 SKIP grams (CBOW meglio per grande corpus)

# Addestrare il modello
#model.train(corpus, total_examples=len(corpus), epochs=10)

In [15]:
# Ottenere l'embedding di una parola
embedding = model.wv['dialis']
print("Embedding di 'parola1':", embedding)

# Trovare parole simili
similar_words = model.wv.most_similar('cancr', topn=5)
print("Parole simili a 'parola1':", similar_words)

Embedding di 'parola1': [  7.393481   -0.4388787  -7.706244    4.0007586  -2.053451    6.67583
   6.6844873 -14.103665   -1.7737978   1.1004465]
Parole simili a 'parola1': [('linfom', 0.9835639595985413), ('cmv', 0.9803005456924438), ('carcinom', 0.9655134677886963), ('broncocostrizion', 0.9509091377258301), ('melanom', 0.9469490647315979)]


In [18]:
model.save('my_vecs_GennaioStemming.p')

In [21]:
#word2Vec_model.save('my_vecs.p')
print(model.wv.most_similar('tumor'))

[('ovar', 0.9585742354393005), ('primit', 0.9454521536827087), ('leucem', 0.9451504349708557), ('posit', 0.9349740743637085), ('carcinom', 0.9342782497406006), ('metastas', 0.9330359697341919), ('cmv', 0.9324724674224854), ('cronic', 0.9319581389427185), ('linfom', 0.9313124418258667), ('cancr', 0.9292378425598145)]


In [19]:
print(model.wv.similarity('tumor', 'cancr'))

0.92923796


OSS: la similarity usata qua è il coseno dell'angolo tra due vettori (Lo ho verificato matematicamente con la formula cos(teta) = <v1,v2>/norm(v1)*norm(v2)